In [ ]:
#Тут константы: итерация 0(особая), входящий поток, значение черной дыры и забивочные переменные х,у для определения весов ребер. Тут граф строится
c = 0
w_In = 20
w_q = -1*w_In
x = 0
y = 0

omegaq = 1
omegaab = 2
omegaas = 10
omegabs = 10
omegabd = 3
omegacd = 1
omegaad = 1

edge1 = ("s", "a", {'weight': omegaas})
edge2 = ("s", "b", {'weight': omegabs})
edge3 = ("q", "a", {'weight': omegaq})
edge4 = ("q", "b", {'weight': omegaq})
edge5 = ("q", "c", {'weight': omegaq})
edge6 = ("q", "d", {'weight': omegaq})
edge7 = ("a", "b", {'weight': omegaab})
edge8 = ("b", "d", {'weight': omegabd})
edge9 = ("c", "d", {'weight': omegacd})
edge10 = ("a", "d", {'weight': omegaad})

In [ ]:
#Тут мы строим репрезентацию графа по edge и смотрим нужные списки вершин, например списки соседей
import matplotlib.pyplot as plt
import networkx as nx

list_to_G = [edge1, edge2, edge3, edge4, edge5, edge6, edge7, edge8, edge9, edge10]
G = nx.Graph()
G.add_edges_from(list_to_G)

list(G.nodes)

['s', 'a', 'b', 'q', 'c', 'd']

In [ ]:
#Здесь определяем вес ребра для случаев первой и прочих итераций. После первой итерации "источник" закрывается.
def omega1(x, y, c, G):
  if G.has_edge(x, y):
    if x == 's':
      omega = 10
    elif y == 's':
      omega = 0
    else:
      omega = G.get_edge_data(x,y)['weight']
  else:
    omega = 0
  return omega

In [ ]:
#Здесь определяем первичные ценности вершин
value_list = []

def w(x):
  if x == list(G.nodes)[0]:
    w = w_In
  elif x == list(G.nodes)[3]:
    w = w_q
  else:
    w = 0
  return w

w('s')

#Строим value_list - список ценностей всех вершин до начала игры
for i in range(len(list(G.nodes))):
  value_list.append(w(list(G.nodes)[i]))

#Определяем сумму файринга для каждой вершины
def total_To_Fire(i, G):
  new_value = 0
  for j in list(G.neighbors(i)):
    currentVal = omega1(i, j, c, G)
    new_value += currentVal
  return new_value

value_list

#Делаем список сумм файринга
firing_list = []
for i in list(G.nodes):
  firing_list.append(total_To_Fire(i, G))

#лист-связка ценность вершины - сумма файринга
fired_firing_list = [(i,j) for i,j in zip(value_list,firing_list)]
#связка вершина - текущая ценность - сумма файринга
vertices_fired_firing_list = [(i,j,k) for i,j,k in zip(G.nodes, value_list,firing_list)]
S_fired_firing_list = sorted(vertices_fired_firing_list, key = lambda x: x[1], reverse = True)

print(value_list)
print(firing_list)
print(vertices_fired_firing_list)
print(S_fired_firing_list)

[20, 0, 0, -20, 0, 0]
[20, 4, 6, 4, 2, 6]
[('s', 20, 20), ('a', 0, 4), ('b', 0, 6), ('q', -20, 4), ('c', 0, 2), ('d', 0, 6)]
[('s', 20, 20), ('a', 0, 4), ('b', 0, 6), ('c', 0, 2), ('d', 0, 6), ('q', -20, 4)]


In [ ]:
for k in range(len(fired_firing_list)):
  print(vertices_fired_firing_list[k])

def vertex_To_Be_Fired(G, vertices_fired_firing_list):
  FFL = vertices_fired_firing_list
  FL = [item for item in FFL if item[1] >= item[2]]
  SFL = sorted(FL, key = lambda x: x[1], reverse = True)
  value_To_Fire = SFL[0][1]
  vertex_To_Fire = SFL[0][0]
  FL.clear()
  return value_To_Fire, vertex_To_Fire

vertex_To_Be_Fired(G, vertices_fired_firing_list)

('s', 20, 20)
('a', 0, 4)
('b', 0, 6)
('q', -20, 4)
('c', 0, 2)
('d', 0, 6)


(20, 's')

In [ ]:
value_To_Fire = vertex_To_Be_Fired(G, vertices_fired_firing_list)[0]
vertex_To_Fire = vertex_To_Be_Fired(G, vertices_fired_firing_list)[1]

print(vertex_To_Fire)
print(list(G.neighbors(vertex_To_Fire)))

s
['a', 'b']


In [ ]:
#Убеждаемся, что за итерация игры. После первой итерации игра к с==0 вернуться не должна.
def check_iter(vertex_To_Fire):
  if vertex_To_Fire == 's':
    c = 0
  else:
    c = 1
  return c

#c = check_iter(vertex_To_Fire)

#Выводим сумму, сколько будем вычитать из ценности vertex_To_Fire
def summ_To_Fire(vertex_To_Fire, G):
  c = check_iter(vertex_To_Fire)
  new_value = 0
  for i in list(G.neighbors(vertex_To_Fire)):
    currentVal = omega1(vertex_To_Fire, i, c, G)
    new_value += currentVal
  return new_value

In [ ]:
#Вычитаем сумму ребер между vertex_To_Fire и ее соседями, задаем, что новая ценность вершины под индексом vertex_To_Fire равна разности ее ценности до fire и весов ребер
def fired_vertex(vertex_To_Fire, value_To_Fire, G):
  a = value_To_Fire - summ_To_Fire(vertex_To_Fire, G)
  return a

#fired_vertex(vertex_To_Fire)
#value_list[index_To_Fire] = fired_vertex(vertex_To_Fire)

In [ ]:
#Тут добавляем каждому соседу vertex_To_Fire ценность, равную весу ребра между ней и vertex_To_Fire
def add(vertex_To_Fire, value_list, G):
  c = check_iter(vertex_To_Fire)
  for i in list(G.neighbors(vertex_To_Fire)):
      b = value_list[list(G.nodes).index(i)] + omega1(vertex_To_Fire, i,c, G)
      value_list[list(G.nodes).index(i)] = b
  return value_list

#add(vertex_To_Fire)

In [ ]:
#Осталось прикрутить условие остановки: если не существует вершины, в которой ценность выше чем сумма файринга, то стоп и принт конечной итерации.
def chip_firing_game(value_list, firing_list, G):
  fired_firing_list = [(i,j) for i,j in zip(value_list,firing_list)]
  vertices_fired_firing_list = [(i,j,k) for i,j,k in zip(G.nodes, value_list,firing_list)]
  cont_Criterion = [item for item in fired_firing_list if item[0]>=item[1]]
  if len(cont_Criterion) >= 1:
    value_To_Fire = vertex_To_Be_Fired(G, vertices_fired_firing_list)[0]
    index_To_Fire = value_list.index(value_To_Fire)
    vertex_To_Fire = vertex_To_Be_Fired(G, vertices_fired_firing_list)[1]

    value_list[index_To_Fire] = fired_vertex(vertex_To_Fire, value_To_Fire, G)
    value_list = add(vertex_To_Fire, value_list, G)
    return value_list, ''
  else:
    return value_list, 'Game over with'

for i in range(5):
  print(value_list)
  value_list, msg = chip_firing_game(value_list, firing_list, G)
  if msg == 'Game over with':
    print(f'{msg}:{value_list}')
    break

[0, 1, 1, -5, 1, 2]
Game over with:[0, 1, 1, -5, 1, 2]
